In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Описание датасета
display(
    Markdown(
        "### Датасет содержит транзакции по кредитным картам, "
        "а также бинарную целевую переменную - Class (0 = обычная, 1 = мошенническая транзакция)."
    )
)


# Загрузка данных
df = pd.read_csv("../Data/creditcard.csv")

print("Размер датасета:", df.shape)
display(df.head())
display(df.info())
display(df.describe())

# Функции для расчета условных вероятностей и Байеса
def conditional_probs(data: pd.DataFrame, col):
    return data.groupby(col, observed=True)["Class"].mean().reset_index(name=f"P(Fraud|{col})")

def bayes_conditional(data: pd.DataFrame, col):
    p_target1 = (data["Class"] == 1).mean()
    p_target0 = 1 - p_target1

    rows = []
    for val in data[col].dropna().unique():
        p_val = (data[col] == val).mean() 
        p_target1_given_val = data.loc[data[col] == val, "Class"].mean()
        p_target0_given_val = 1 - p_target1_given_val

        # формула Байеса
        p_val_given_target1 = (p_target1_given_val * p_val) / p_target1
        p_val_given_target0 = (p_target0_given_val * p_val) / p_target0

        rows.append([val, p_val_given_target1, p_val_given_target0])

    return pd.DataFrame(
        rows,
        columns=[col, f"P({col}|Fraud=1)", f"P({col}|Fraud=0)"],
    )

# Непрерывные признаки (Amount, Time) разбиваем на интервалы (бины)
df_grouped = pd.DataFrame(df)

df_grouped["Amount_bin"] = pd.cut(
    df["Amount"],
    bins=[0, 10, 50, 200, 1000, df["Amount"].max() + 1],
    labels=["0–10 $", "11–50 $", "51–200 $", "201–1000 $", "1000+ $"],
    include_lowest=True
)

df_grouped["Time_bin"] = pd.cut(
    df["Time"],
    bins=[0, 21600, 43200, 64800, 86400, df["Time"].max() + 1],
    labels=["0–6 ч", "6–12 ч", "12–18 ч", "18–24 ч", "24+ ч"],
    include_lowest=True
)

display(df_grouped.head())

# Значения показывают условные вероятности P(Fraud|X) для разных категорий
display(Markdown("### Вероятность мошенничества по сумме транзакции"))
display(conditional_probs(df_grouped, "Amount_bin"))

display(Markdown("### Вероятность мошенничества по времени транзакции"))
display(conditional_probs(df_grouped, "Time_bin"))

# Значения показывают условные вероятности P(X|Fraud) по формуле Байеса
display(Markdown("### Вероятность суммы по формуле Байеса"))
display(bayes_conditional(df_grouped, "Amount_bin"))

display(Markdown("### Вероятность времени по формуле Байеса"))
display(bayes_conditional(df_grouped, "Time_bin"))

# Барплоты показывают, как изменяется вероятность мошенничества в зависимости от признака
for col, label in zip(["Amount_bin", "Time_bin"], ["Сумма транзакции", "Время транзакции"]):
    plt.figure(figsize=(6, 4))
    sns.barplot(
        x=col, y="Class", data=df_grouped, estimator=np.mean, errorbar=None
    )
    plt.title(f"Вероятность мошенничества в зависимости от {label}")
    plt.xlabel(label)
    plt.ylabel("Вероятность мошенничества")
    plt.show()

# Основные выводы
display(
    Markdown(
        """### Основные выводы:

- Мошеннические транзакции крайне редки (~0.17% всех транзакций).  

- Вероятность мошенничества растет с увеличением суммы транзакции.  

- Некоторые временные интервалы (ночь и раннее утро) показывают чуть более высокую вероятность мошенничества."""
    )
)


### Датасет содержит транзакции по кредитным картам, а также бинарную целевую переменную - Class (0 = обычная, 1 = мошенническая транзакция).

FileNotFoundError: [Errno 2] No such file or directory: '../Data/creditcard.csv'